In [30]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely import wkb
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors

import numpy as np
from scipy import stats
import math
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine

In [56]:
#read in groud truth data
gt = pd.read_csv('../../data/user-study/chi-24-jon-labels-crowdstudy.csv')
gt

,username,label_id,route_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,severity,temporary,tag_list,description
0,jonfroehlich,116,1,27789,East Oradell,CurbRamp,40.953232,-74.017944,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:06.517-07,1.0,f,NaN,NaN
1,jonfroehlich,117,1,27789,East Oradell,Crosswalk,40.953285,-74.017929,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:10.8-07,1.0,f,NaN,NaN
2,jonfroehlich,118,1,27789,East Oradell,CurbRamp,40.953339,-74.017952,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:12.327-07,1.0,f,NaN,NaN
3,jonfroehlich,119,1,27959,East Oradell,NoSidewalk,40.953117,-74.018028,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:18.249-07,NaN,f,NaN,NaN
4,jonfroehlich,120,1,27959,East Oradell,NoSidewalk,40.953171,-74.018074,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:19.668-07,NaN,f,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,jonfroehlich,643,6,7062,Bryant,NoCurbRamp,47.664955,-122.287888,KnZrYzS8CnsQP7MamK6u9g,2023-08-04 06:47:06.422-07,5.0,f,no alternate route,NaN
460,jonfroehlich,644,6,7062,Bryant,SurfaceProblem,47.664951,-122.287888,KnZrYzS8CnsQP7MamK6u9g,2023-08-04 06:47:13.128-07,2.0,f,"height difference,uneven/slanted",NaN
461,jonfroehlich,645,6,7062,Bryant,NoCurbRamp,47.664967,-122.288010,KnZrYzS8CnsQP7MamK6u9g,2023-08-04 06:47:22.358-07,5.0,f,no alternate route,NaN
462,jonfroehlich,646,6,4147,Bryant,NoCurbRamp,47.664936,-122.288040,KnZrYzS8CnsQP7MamK6u9g,2023-08-04 06:47:27.606-07,5.0,f,no alternate route,NaN


In [57]:
#select only the main label_types
gt = gt[gt['label_type'].isin(['CurbRamp', 'NoCurbRamp', 'Obstacle', 'SurfaceProblem','NoSidewalk'])]
gt

,username,label_id,route_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,severity,temporary,tag_list,description
0,jonfroehlich,116,1,27789,East Oradell,CurbRamp,40.953232,-74.017944,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:06.517-07,1.0,f,NaN,NaN
2,jonfroehlich,118,1,27789,East Oradell,CurbRamp,40.953339,-74.017952,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:12.327-07,1.0,f,NaN,NaN
3,jonfroehlich,119,1,27959,East Oradell,NoSidewalk,40.953117,-74.018028,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:18.249-07,NaN,f,NaN,NaN
4,jonfroehlich,120,1,27959,East Oradell,NoSidewalk,40.953171,-74.018074,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:19.668-07,NaN,f,NaN,NaN
5,jonfroehlich,122,1,27961,East Oradell,NoSidewalk,40.953232,-74.018097,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:22.916-07,NaN,f,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,jonfroehlich,643,6,7062,Bryant,NoCurbRamp,47.664955,-122.287888,KnZrYzS8CnsQP7MamK6u9g,2023-08-04 06:47:06.422-07,5.0,f,no alternate route,NaN
460,jonfroehlich,644,6,7062,Bryant,SurfaceProblem,47.664951,-122.287888,KnZrYzS8CnsQP7MamK6u9g,2023-08-04 06:47:13.128-07,2.0,f,"height difference,uneven/slanted",NaN
461,jonfroehlich,645,6,7062,Bryant,NoCurbRamp,47.664967,-122.288010,KnZrYzS8CnsQP7MamK6u9g,2023-08-04 06:47:22.358-07,5.0,f,no alternate route,NaN
462,jonfroehlich,646,6,4147,Bryant,NoCurbRamp,47.664936,-122.288040,KnZrYzS8CnsQP7MamK6u9g,2023-08-04 06:47:27.606-07,5.0,f,no alternate route,NaN


In [59]:
gt['label_type'].value_counts()

SurfaceProblem    252
NoSidewalk        101
NoCurbRamp         40
Obstacle           33
CurbRamp           19
Name: label_type, dtype: int64

In [60]:
#split into two cities
oradell = gt[gt['route_id'] < 5]
seattle = gt[gt['route_id'] > 4]


In [62]:
def custom_dist(u, v):
#     if u[2] == v[2]:
#         return sys.float_info.max
#     else:
        return haversine([u[0], u[1]], [v[0], v[1]])

thresholds = {'CurbRamp': 0.0075,
                      'NoCurbRamp': 0.0075,
                      'SurfaceProblem': 0.01,
                      'Obstacle': 0.01,
                      'NoSidewalk': 0.01,
                      'Crosswalk': 0.01,
                      'Signal': 0.01,
                      'Occlusion': 0.01,
                      'Other': 0.01,
                      'Problem': 0.01}

def cluster(labels, label_type, thresholds):

    # Makes a normal dist matrix for a single user, but uses special dist function for multi-user clustering that
    # prevents the same user's attributes from being clustered together.
    #dist_matrix = pdist(np.array(labels[['lat', 'lng', 'user_id']].values), custom_dist)
    dist_matrix = pdist(np.array(labels[['lat', 'lng']].values), custom_dist)
    link = linkage(dist_matrix, method='complete')
    curr_type = labels['label_type'].values

    # Copies the labels dataframe and adds a column to it for the cluster id each label is in.
    labelsCopy = labels.copy()
    labelsCopy.loc[:,'cluster_id'] = fcluster(link, t=thresholds[label_type], criterion='distance')

    # Cuts tree so that only labels less than clust_threth kilometers apart are clustered.
    clusters = labelsCopy.groupby('cluster_id')

    # Computes the center of each cluster and assigns temporariness and severity.
    cluster_list = [] # list of tuples (label_type, cluster_num, lat, lng, severity, temporary).
    for clust_num, clust in clusters:
        #ave_pos = np.mean(clust['coords'].tolist(), axis=0) # use ave pos of clusters.
        #ave_sev = None if pd.isnull(clust['severity']).all() else int(round(np.median(clust['severity'][~np.isnan(clust['severity'])])))
        #ave_temp = None if pd.isnull(clust['temporary']).all() else bool(round(np.mean(clust['temporary'])))

        cluster_list.append((curr_type, clust_num))

    cluster_df = pd.DataFrame(cluster_list, columns=['label_type', 'cluster_id'])

    return (cluster_df, labelsCopy)


# Pick which label types should be included in clustering, and which should be included in the "Problem" type.
label_types = ['CurbRamp', 'NoSidewalk', 'Occlusion', 'SurfaceProblem', 'Obstacle', 'Other', 'NoCurbRamp', 'Crosswalk', 'Signal']
# problem_types = ['SurfaceProblem', 'Obstacle']

# These are the columns required in the POST requests for the labels and clusters, respectively.
label_cols = ['label_id', 'label_type', 'cluster_id']
cluster_cols = ['label_type', 'cluster_id', 'lat', 'lng', 'severity', 'temporary']


def cluster_label_type_at_index(label_data,i):
        clusters_for_type_i = pd.DataFrame(columns=cluster_cols)
        labels_for_type_i = pd.DataFrame(columns=label_cols)

        label_type = label_types[i]
        # if label_type == 'Problem':
        #     type_data = label_data[label_data.label_type.isin(problem_types)]
        # else:
        type_data = label_data[label_data.label_type == label_type]

        # If there are >1 labels, we can do clustering. Otherwise just copy the 1 (or 0) labels.
        if type_data.shape[0] > 1:
            (clusters_for_type_i, labels_for_type_i) = cluster(type_data, label_type, thresholds)
        elif type_data.shape[0] == 1:
            labels_for_type_i = type_data.copy()
            labels_for_type_i.loc[:,'cluster_id'] = 1 # Gives the single cluster a cluster_id of 1.
            labels_for_type_i.loc[:,'label_type'] = label_type # Gives Problem type if needed.
            clusters_for_type_i = labels_for_type_i.filter(items=cluster_cols)

        return (label_type, clusters_for_type_i, labels_for_type_i)

In [98]:
#replace labels with clusters
def replace_labels_with_clusters(df,label_type_index):
    label_types = ['CurbRamp', 'NoSidewalk', 'Occlusion', 'SurfaceProblem', 'Obstacle', 'Other', 'NoCurbRamp', 'Crosswalk', 'Signal']
    #select the label type we want to cluster
    df1 = df[df['label_type'] == label_types[label_type_index]]
    #run clustering algorithm
    df2 = cluster_label_type_at_index(df1,label_type_index)[2]
    #change nan severity to 0
    df2['severity'] = df2['severity'].fillna(0)
    #keep labels with highest severity within each cluster
    df3= df2.loc[df2.groupby('cluster_id')['severity'].idxmax()]
    #drop cluster_id column
    df3 = df3.drop(columns=['cluster_id'])
    #delete label type rows from original df
    df4 = df.drop(df[df['label_type'] == label_types[label_type_index]].index)
    #append new labels
    df5 = df4.append(df3)
    #sort by route_id and street_edge_id
    df5 = df5.sort_values(by=['route_id','street_edge_id'])
    return df5

In [99]:
oradell_cleaned = replace_labels_with_clusters(oradell,3)
oradell_cleaned = replace_labels_with_clusters(oradell_cleaned,1)
seattle_cleaned = replace_labels_with_clusters(seattle,3)

In [100]:
oradell_cleaned

,username,label_id,route_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,severity,temporary,tag_list,description
93,jonfroehlich,211,1,27655,East Oradell,CurbRamp,40.953526,-74.014587,K91xNhf5hU65AqkTVBxtlw,2023-08-02 16:41:58.961-07,2.0,f,points into traffic,NaN
95,jonfroehlich,212,1,27655,East Oradell,CurbRamp,40.953495,-74.014465,K91xNhf5hU65AqkTVBxtlw,2023-08-02 16:42:03.441-07,2.0,f,points into traffic,NaN
0,jonfroehlich,116,1,27789,East Oradell,CurbRamp,40.953232,-74.017944,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:06.517-07,1.0,f,NaN,NaN
2,jonfroehlich,118,1,27789,East Oradell,CurbRamp,40.953339,-74.017952,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:12.327-07,1.0,f,NaN,NaN
35,jonfroehlich,152,1,27789,East Oradell,Obstacle,40.953426,-74.017059,vFHIBHYHF5vNqC4jJvlCWA,2023-08-02 16:33:46.414-07,2.0,t,trash/recycling can,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,jonfroehlich,535,4,27982,West Oradell,NoSidewalk,40.955582,-74.042358,Wcm6bWuJhgEyVj9VNJ_UXw,2023-08-04 05:55:12.651-07,0.0,f,NaN,NaN
335,jonfroehlich,534,4,27982,West Oradell,NoSidewalk,40.955669,-74.042480,Wcm6bWuJhgEyVj9VNJ_UXw,2023-08-04 05:55:11.833-07,0.0,f,NaN,NaN
339,jonfroehlich,538,4,27982,West Oradell,NoSidewalk,40.955750,-74.042442,ZH9HxC1NBaQLPIdE4hv6aw,2023-08-04 05:55:17.836-07,0.0,f,NaN,NaN
321,jonfroehlich,545,4,27982,West Oradell,NoSidewalk,40.955860,-74.042389,aVzRcV_Pkh2aWDLUT2rTIg,2023-08-04 05:55:27.342-07,0.0,f,NaN,NaN


In [143]:
#group by street_edge_id and label_type
oradell_cleaned_per_route = oradell_cleaned.groupby(['route_id','street_edge_id','label_type']).size().to_frame('count')
seattle_cleaned_per_route = seattle_cleaned.groupby(['route_id','street_edge_id','label_type']).size().to_frame('count')

In [144]:
#append oradell and seattle
total_cleaned_per_route = oradell_cleaned_per_route.append(seattle_cleaned_per_route)
#save to excel
total_cleaned_per_route.to_excel('../../data/user-study/total_cleaned_per_route.xlsx', index=True)

In [146]:
a = oradell['label_type'].value_counts().reindex(['CurbRamp','NoCurbRamp','Obstacle','SurfaceProblem','NoSidewalk'])

In [147]:
b = oradell_cleaned['label_type'].value_counts().reindex(['CurbRamp','NoCurbRamp','Obstacle','SurfaceProblem','NoSidewalk'])

In [150]:
c = seattle['label_type'].value_counts().reindex(['CurbRamp','NoCurbRamp','Obstacle','SurfaceProblem','NoSidewalk'])

In [151]:
d = seattle_cleaned['label_type'].value_counts().reindex(['CurbRamp','NoCurbRamp','Obstacle','SurfaceProblem','NoSidewalk'])

In [154]:
#make a and b into a dataframe
df = pd.DataFrame({'raw_labels_oradell': a, 'clustered_oradell': b, 'raw_labels_seattle': c, 'clustered_seattle': d})
#change nan to 0 and convert to int
df = df.fillna(0).astype(int)
#add total column
df['total_raw_labels'] = df['raw_labels_oradell'] + df['raw_labels_seattle']
df['total_clustered'] = df['clustered_oradell'] + df['clustered_seattle']
df

,raw_labels_oradell,clustered_oradell,raw_labels_seattle,clustered_seattle,total_raw_labels,total_clustered
CurbRamp,17,17,2,2,19,19
NoCurbRamp,27,27,13,13,40,40
Obstacle,11,11,22,22,33,33
SurfaceProblem,218,84,34,19,252,103
NoSidewalk,101,41,0,0,101,41


In [155]:
#save df to excel
df.to_excel('../../data/user-study/label_counts_summary.xlsx', index=True)

In [190]:
#read in routes data
routes = pd.read_csv('../../data/user-study/chi-24-routes-crowdstudy.csv')

In [191]:
#translate routes into geodataframe using geom
routes['geom'] = routes['geom'].apply(wkb.loads, hex=True)
routes = gpd.GeoDataFrame(routes, geometry='geom')
routes.crs = 'epsg:4326'
routes

,route_id,region_id,street_edge_id,way_type,geom
0,1,96,27789,residential,"LINESTRING (-74.01450 40.95343, -74.01681 40.9..."
1,1,96,27792,residential,"LINESTRING (-74.01357 40.95340, -74.01374 40.9..."
2,2,95,27992,residential,"LINESTRING (-74.04594 40.96277, -74.04605 40.9..."
3,2,95,27683,residential,"LINESTRING (-74.04630 40.96198, -74.04629 40.9..."
4,2,95,27906,residential,"LINESTRING (-74.04389 40.96133, -74.04630 40.9..."
5,3,94,27968,residential,"LINESTRING (-74.04096 40.95473, -74.04051 40.9..."
6,3,94,27969,residential,"LINESTRING (-74.04171 40.95495, -74.04096 40.9..."
7,4,95,27970,residential,"LINESTRING (-74.04259 40.95521, -74.04192 40.9..."
8,4,95,27982,residential,"LINESTRING (-74.04259 40.95521, -74.04226 40.9..."
9,5,30,7555,residential,"LINESTRING (-122.28520 47.60683, -122.28499 47..."


In [207]:
#separate into two cities
oradell_routes = routes[routes['route_id'] < 5]
seattle_routes = routes[routes['route_id'] > 4]

In [208]:
#change crs for seattle and oradell 
oradell_routes = oradell_routes.to_crs('epsg:3424')
seattle_routes = seattle_routes.to_crs('epsg:2285')

In [209]:
#add distance column in miles and round to 2 decimal places
oradell_routes['distance_mi'] = oradell_routes['geom'].length * 0.000621371
oradell_routes['distance_mi'] = oradell_routes['distance_mi'].round(2)
seattle_routes['distance_mi'] = seattle_routes['geom'].length * 0.000621371
seattle_routes['distance_mi'] = seattle_routes['distance_mi'].round(2)


In [210]:
oradell_routes

,route_id,region_id,street_edge_id,way_type,geom,distance_mi
0,1,96,27789,residential,"LINESTRING (626220.287 772603.992, 625583.336 ...",0.61
1,1,96,27792,residential,"LINESTRING (626477.488 772595.221, 626429.630 ...",0.16
2,2,95,27992,residential,"LINESTRING (617518.829 775959.973, 617487.010 ...",0.22
3,2,95,27683,residential,"LINESTRING (617422.009 775671.656, 617423.031 ...",0.19
4,2,95,27906,residential,"LINESTRING (618088.527 775438.682, 617422.009 ...",0.44
5,3,94,27968,residential,"LINESTRING (618908.881 773035.471, 619034.099 ...",0.14
6,3,94,27969,residential,"LINESTRING (618700.684 773116.626, 618908.881 ...",0.14
7,4,95,27970,residential,"LINESTRING (618457.875 773208.381, 618643.376 ...",0.16
8,4,95,27982,residential,"LINESTRING (618457.875 773208.381, 618548.045 ...",0.18


In [187]:
#put oradell_cleaned into geodataframe
oradell_cleaned['geom'] = oradell_cleaned.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)
oradell_cleaned = gpd.GeoDataFrame(oradell_cleaned, geometry='geom')
oradell_cleaned.crs = 'epsg:4326'
oradell_cleaned.head()

,username,label_id,route_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,severity,temporary,tag_list,description,geom,distance
93,jonfroehlich,211,1,27655,East Oradell,CurbRamp,40.953526,-74.014587,K91xNhf5hU65AqkTVBxtlw,2023-08-02 16:41:58.961-07,2.0,f,points into traffic,NaN,POINT (-74.01459 40.95353),0.000302
95,jonfroehlich,212,1,27655,East Oradell,CurbRamp,40.953495,-74.014465,K91xNhf5hU65AqkTVBxtlw,2023-08-02 16:42:03.441-07,2.0,f,points into traffic,NaN,POINT (-74.01447 40.95350),0.000233
0,jonfroehlich,116,1,27789,East Oradell,CurbRamp,40.953232,-74.017944,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:06.517-07,1.0,f,NaN,NaN,POINT (-74.01794 40.95323),0.000091
2,jonfroehlich,118,1,27789,East Oradell,CurbRamp,40.953339,-74.017952,tw7_pOjIeJmDZOssQNeh7A,2023-08-02 16:29:12.327-07,1.0,f,NaN,NaN,POINT (-74.01795 40.95334),0.000254
35,jonfroehlich,152,1,27789,East Oradell,Obstacle,40.953426,-74.017059,vFHIBHYHF5vNqC4jJvlCWA,2023-08-02 16:33:46.414-07,2.0,t,trash/recycling can,NaN,POINT (-74.01706 40.95343),0.000077


In [184]:
#put seattle_cleaned into geodataframe
seattle_cleaned['geom'] = seattle_cleaned.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)
seattle_cleaned = gpd.GeoDataFrame(seattle_cleaned, geometry='geom')
seattle_cleaned.crs = 'epsg:4326'
seattle_cleaned.head()

,username,label_id,route_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,severity,temporary,tag_list,description,geom
385,jonfroehlich,568,5,7555,Leschi,Obstacle,47.607956,-122.284920,AeTEoqPeBsj68TBGSjVgvw,2023-08-04 06:25:59.547-07,5.0,f,trash/recycling can,NaN,POINT (-122.28492 47.60796)
386,jonfroehlich,569,5,7555,Leschi,Obstacle,47.607925,-122.284920,AeTEoqPeBsj68TBGSjVgvw,2023-08-04 06:26:06.817-07,5.0,f,trash/recycling can,NaN,POINT (-122.28492 47.60793)
387,jonfroehlich,570,5,7555,Leschi,Obstacle,47.607872,-122.284821,AeTEoqPeBsj68TBGSjVgvw,2023-08-04 06:26:14.956-07,5.0,f,fire hydrant,NaN,POINT (-122.28482 47.60787)
388,jonfroehlich,571,5,7555,Leschi,Obstacle,47.607849,-122.284813,AeTEoqPeBsj68TBGSjVgvw,2023-08-04 06:26:20.145-07,NaN,f,trash/recycling can,NaN,POINT (-122.28481 47.60785)
389,jonfroehlich,572,5,7555,Leschi,Obstacle,47.607899,-122.284836,AeTEoqPeBsj68TBGSjVgvw,2023-08-04 06:26:26.498-07,5.0,f,"vegetation,narrow",NaN,POINT (-122.28484 47.60790)


In [173]:
label_colors ={'Crosswalk':'#FABF1C',
               'CurbRamp':'#90C31F',
               'NoCurbRamp':'#E679B6',
               'NoSidewalk':'#BE87D8',
               'Obstacle' :'#78B0EA',
               'Occlusion':'#B3B3B3',
               'Other':'#B3B3B3',
               'Signal':'#63C0AB',
               'SurfaceProblem':'#F68D3E'}

In [174]:
def get_label_colors(geof):
    label_list = geof['label_type'].unique().tolist()
    label_list.sort()
    j = []
    for i in label_list:
        j.append(label_colors[i])
    return j

In [212]:
oradell_routes_map = oradell_routes.explore(tiles = 'cartodbpositron',  tooltip = ["route_id", "street_edge_id",'distance_mi'], column ='street_edge_id', categorical=True)

In [216]:
 oradell_routes_and_labels = oradell_cleaned.explore( m = oradell_routes_map, tiles='cartodbpositron', tooltip=['label_id','street_edge_id','route_id'],popup=['label_id','street_edge_id','route_id'],  column= 'label_type',cmap=get_label_colors(oradell_cleaned))

In [217]:
seattle_routes_map = seattle_routes.explore(tiles = 'cartodbpositron',  tooltip = ["route_id", "street_edge_id",'distance_mi'], column ='street_edge_id', categorical=True)

In [218]:
seattle_routes_and_labels = seattle_cleaned.explore( m = seattle_routes_map, tiles='cartodbpositron', tooltip=['label_id','street_edge_id','route_id'],popup=['label_id','street_edge_id','route_id'],  column= 'label_type',cmap=get_label_colors(seattle_cleaned))

In [219]:
oradell_routes_and_labels.save('../../maps/user-study/oradell_routes_and_labels.html')
seattle_routes_and_labels.save('../../maps/user-study/seattle_routes_and_labels.html')